In [ ]:
!pip install laceworksdk requests~=2.25 urllib3~=1.26

In [ ]:
LW_ACCOUNT=""
LW_API_KEY=""
LW_API_SECRET=""

In [ ]:
from datetime import datetime, timedelta, timezone
from pprint import pprint

from laceworksdk import LaceworkClient

# Build start/end times
current_time = datetime.now(timezone.utc)
start_time = current_time - timedelta(days=1)
start_time = start_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")
end_time = current_time.strftime("%Y-%m-%dT%H:%M:%S.000Z")

lw = LaceworkClient(account=LW_ACCOUNT,
                    api_key=LW_API_KEY,
                    api_secret=LW_API_SECRET)

lw.set_org_level_access(True)

user_profile = lw.user_profile.get()
user_profile_data = user_profile.get('data', {})[0]

lw.set_org_level_access(False)

results = []

if user_profile_data.get('orgAccount', False):

    for subaccount in user_profile_data.get('accounts', []):
        
        lw.set_subaccount(subaccount['accountName'])

        response = lw.queries.execute(
            query_text="""LW_Custom_Host_CVE_2022_0185 {
                source {
                    LW_HE_MACHINES
                }
                filter {
                    OS = 'Amazon Linux'
                    and OS_VERSION = '2'
                    and to_double(substring(KERNEL_RELEASE, 1, 1)) = 5
                    and to_double(substring(KERNEL_RELEASE, 3, 1)) >= 1
                    and to_double(substring(KERNEL_RELEASE, 3, 1)) <= 4
                    and to_double(substring(KERNEL_RELEASE, 5, 3)) < 172
                }
                return {
                    MID,
                    HOSTNAME,
                    KERNEL,
                    KERNEL_RELEASE,
                    KERNEL_VERSION,
                    OS,
                    OS_VERSION,
                    TAGS,
                    TAGS:"eks:cluster-name"::String as CLUSTER_NAME,
                    TAGS:"eks:nodegroup-name"::String as CLUSTER_NODE_GROUP,
                    TAGS:Account::String as ACCOUNT,
                    TAGS:Zone::String as ZONE,
                    TAGS:InstanceId::String as INSTANCE
                }
            }""",
            arguments={
                "StartTimeRange": start_time,
                "EndTimeRange": end_time
            }
        )

        for machine in response["data"]:
          results.append(machine)

In [ ]:
print(len(results))

In [ ]:
import pandas as pd
df = pd.DataFrame(results)
df.head(10000)